In [1]:
import json
import argparse
import numpy as np

import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.utils.data.distributed
from torch.utils.data import DataLoader
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
from torchvision import transforms

from base.utils import Logger, TBDiskWriter, setup_train_config, load_decomposition, load_primals
from base.modules import weights_init

from LF.models import Generator, Discriminator, AuxClassifier
from LF.phase1_dataset import LF1TrainDataset, LF1TestDataset
from LF.phase1_trainer import LF1Trainer
from LF.phase2_dataset import LF2TrainDataset, LF2TestDataset
from LF.phase2_trainer import LF2Trainer


TRANSFORM = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

PHASE_DICT = {
    1: (LF1TrainDataset, LF1TestDataset, LF1Trainer),
    2: (LF2TrainDataset, LF2TestDataset, LF2Trainer),
}

